In [7]:
import xlwings as xw
import pandas as pd
import difflib
import re
import os 
import numpy as np

In [8]:
# ----------------------------------- 文件预处理 ------------------------------------ 
def prepocess(path,app):
    wb = app.books.open(path)
    #表格总数
    sht_ai =  list(range(0,len(wb.sheets)))  
    

    for sht_i in sht_ai:
        #1.除去空白页 ————删除页签造成sht_ai超出实际页签个数 
        sht = wb.sheets[sht_i]
        #取表格使用范围内的值
        sht_value = sht.range((1,1),sht.used_range.shape).value
        if sht_value == None:
            sht_ai.pop()
            sht.delete()
            wb.save(path)
        #2.只保留含有周数据的表格
        
        
        #3.去除隐藏列(暂未实现)
    wb.close()
    


In [9]:
# ------------------------------------提取详细数据------------------------------------ 
def total_a(path,name,app,j):
    wb = app.books.open(path)
    # #表格总数
    sht_ai =  len(wb.sheets)

    #sht_aname 取出所有表名
    t = [l for l in wb.sheets]
    sht_aname = list()
    for i in range(0,sht_ai):
        sht_aname.append(t[i].name)
    #sht_aname_index 取出周工作表序号的bool list
    sht_aname_index = list()
    for i in sht_aname:
        if "周工作" in i: sht_aname_index.append(True)
        else:sht_aname_index.append(False)


    t = [i for i in range(0,sht_ai)]
    t = np.array(t)[np.array(sht_aname_index)]


    #当前表格
    cur = 1

    # for sht_i in range(1,sht_ai):
    for sht_i in t:
    
        sht = wb.sheets[sht_i]
        #取表格使用范围
        # sht_value = sht.range('A1:T30').value
        sht_value = sht.range((1,1),sht.used_range.shape).value
        #取得包含任务序号的第一列
        first_col_list  = [i[0] for i in sht_value]
        frnum = first_col_list.index('任务编号') + 1
        #正则找到第一列小计元素
        first_col_list = [str(v) for v in first_col_list]
        resr = difflib.get_close_matches('小计', first_col_list, 1, cutoff=0.5)
        #索引该元素
        lrnum = first_col_list.index(resr[0]) 

        # num_list_right =  list(filter(None, first_col_list[frnum:lrnum])) 
        # wnum = len(num_list_right) #任务数
        rnum = lrnum - frnum #行数

        #表头行的None值转化为字符形式
        first_row_list  = sht_value[1]
        first_row_list = ['None' if v is None else v for v in first_row_list]
        #正则找到项目用时统计的元素位置
        resc = difflib.get_close_matches('项目用时统计', first_row_list, 1, cutoff=0.5)
        #索引该元素得到
        cnum = sht_value[1].index(resc[0]) #列数

        #数据框提取
        worktemp = pd.DataFrame(sht_value[frnum-1:lrnum+1])
        worktemp = worktemp.iloc[:,0:(cnum+1)]
        rowname = worktemp.iloc[0]
        worktemp = worktemp.rename(columns=rowname).iloc[1:]
        worktemp.columns = [*worktemp.columns[:-1], '项目投入统计']
        worktemp = worktemp.reset_index(drop=True)
    
        
        sum_st_col = worktemp.columns.get_loc('星期一')
        sum_sp_col = cnum 
        daynum = sum_sp_col - sum_st_col

        # # 数据矫正
        worktemp['项目投入统计'] = worktemp.iloc[:,sum_st_col:sum_sp_col].sum(axis=1)
        if (worktemp['项目投入统计'][0:-1].sum()) == (worktemp['项目投入统计'].iloc[-1]):
            # print('项目投入统计正确')
            pass
        # #工作天未自增1h
        # elif ((worktemp['项目投入统计'].iloc[-1]) - worktemp['项目投入统计'][0:-1].sum()) == daynum:
        #     # worktemp['项目投入统计'].iloc[:-1] = worktemp['项目投入统计'].iloc[:-1] + (daynum/rnum)
        #     # worktemp['项目投入统计'].iloc[-1] = worktemp['项目投入统计'].iloc[-1] + daynum
        #     print(name[0]+'项目投入统计数值在第'+ str(sht_i) +'周出错（加和原因）')
        else:
            print(name[0]+'项目投入统计数值在第'+ str(sht_i) +'周出错（填报原因）')



        if os.path.isfile("./汇总明细.xlsx"):

            wb0 = xw.books.open ( './汇总明细.xlsx' )
            wb0.sheets[j].name = name[0]
            wb0.sheets[j].range(cur,1).expand().value = worktemp.values.tolist()
            wb0.save()

        else:
            wb0 = xw.Book()
            wb0.sheets[0].name = name[0]
            wb0.sheets[0].range(cur,1).expand().value = worktemp.values.tolist()
            wb0.save('./汇总明细.xlsx')
            wb0.close()



        cur = cur + rnum + 1  
    app.quit()


In [10]:
# ------------------------------------汇总数据处理------------------------------------ 
def total_b(app):
    if os.path.isfile("./汇总.xlsx"):
        app = xw.App(visible=True,add_book=False)
        # app.display_alerts=False
        # app.screen_updating=False
        wb = app.books.open('./汇总.xlsx')
    else:
        os.system('copy 汇总明细.xlsx 汇总.xlsx')
        app = xw.App(visible=True,add_book=False)
        # app.display_alerts=False
        # app.screen_updating=False
        wb = app.books.open('./汇总.xlsx')



    #删除小计




    #向上合并任务






    app.quit()


In [11]:
# ------------------------------------ 主程序入口 ------------------------------------ 
def main():
    #------------全局变量-------------------------------------------------
    #将文件夹中所有excel路径存入list
    p = os.getcwd()
    fl = os.listdir('./input')                                     #CH
    l = [i for i in fl if '双周计划' in i]                          #CH
    # l.remove('汇总.xlsx')
    l = [p+"\\input\\"+i for i in l]
    #
    j = 0 #汇总明细excel页签flag

    #---------------------------------------------------------------------

    #循环从excel文档去除空白页 提取明细数据
    for g in l:
        path = g
        name = re.findall(' - (.*)\.',path)                        #CH
        app = xw.App(visible=True,add_book=False)
        # app.display_alerts=False
        # app.screen_updating=False
        # prepocess(path,app)
        total_a(path,name,app,j)
        j += 1

    # # ------------汇总数据处理------------- 


    # app = xw.App(visible=True,add_book=False)
    #     # app.display_alerts=False
    #     # app.screen_updating=False

In [12]:
# ------------------------------------主程序调用------------------------------------ 
main()

符芳恺项目投入统计数值在第1周出错（填报原因）
符芳恺项目投入统计数值在第2周出错（填报原因）
符芳恺项目投入统计数值在第3周出错（填报原因）
符芳恺项目投入统计数值在第4周出错（填报原因）


C:\Users\wengzhenbin\AppData\Local\Temp\ipykernel_7900\1180619563.py:68: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  worktemp['项目投入统计'] = worktemp.iloc[:,sum_st_col:sum_sp_col].sum(axis=1)
C:\Users\wengzhenbin\AppData\Local\Temp\ipykernel_7900\1180619563.py:68: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  worktemp['项目投入统计'] = worktemp.iloc[:,sum_st_col:sum_sp_col].sum(axis=1)


In [13]:
import datetime
datetime.datetime.now().strftime('%y%m%d%H%M%S')

'220610212526'